In [ ]:
########################################################
#
#     Title: RAW [EXTRACT] - HaloPSA [API]
#     Created by: Gabe McWilliams
#     Date: 2023/04/19
#
########################################################

# Data Shaping
import pandas as pd
import re
import datetime as dt
import hvac

# API Calls
import requests
from requests.structures import CaseInsensitiveDict

# File Handling and Export
import json


class HaloPSA:

    def __init__(self):
        self.__secrets = self.read_secret()
        self.__access_token = self.__create_token()

    @staticmethod
    # def read_secret(mount_point="api", path="halo_psa/dev/acronis"):
    def read_secret(mount_point="api", path="HALO_PSA"):
        client = hvac.Client()
        resp = client.kv.v2.read_secret(mount_point=mount_point, path=f'/{path}')
        secret = resp['data']['data']

        return secret

    def __create_token(self):
        try:
            # call token api url
            token_uri = f"{self.__secrets['base_uri']}/auth/token?tenant={self.__secrets['tenant']}"

            # construct header
            headers = CaseInsensitiveDict()
            headers['Content-Type'] = 'application/x-www-form-urlencoded'

            # construct req body
            data = CaseInsensitiveDict()
            data['grant_type'] = 'client_credentials'
            data['client_id'] = self.__secrets['client_id']
            data['client_secret'] = self.__secrets['client_secret']
            data['scope'] = 'all'

            # request content response
            resp = requests.post(token_uri, headers=headers, data=data)
            content = resp.content.decode('utf-8')
            c_dict = json.loads(content)

            return {
                "data": c_dict["access_token"],
                "result": {
                    "status_code": 200,
                    "message": "Success"
                }
            }

        except Exception as e:
            return {
                "result": {
                    "status_code": 500,
                    "message": str(e)
                }
            }
        

    def create_ticket_type_dataframe(self):


        # request content response
        request_url = f"{self.__secrets['base_uri']}/api/TicketType"
        print(request_url)

        # construct header
        headers = CaseInsensitiveDict()
        headers['Authorization'] = f'Bearer {self.__access_token}'
        headers['Content-Type'] = 'application/json'

        # construct req body
        data = ''

        print(f'Request URL: {request_url}')

        resp = requests.get(request_url, headers=headers, data=data)
        # print(headers)
        print(resp.text)
        content = resp.content
        
        print(content)
        c_dict = json.loads(content)
        df = pd.DataFrame(c_dict)

        return {
            "data": df,
            "result": {

                "status_code": 200,
                "status_message": "Dataframe created successfully"
            }
        }
       

In [ ]:
halo = HaloPSA()

In [ ]:
halo.create_ticket_type_dataframe()

In [ ]:
halo.create_sites_dataframe()

In [ ]:
df = pd.read_parquet("./api_2023_09_07_104559_halo_psa_tickets.parquet", dtype_backend="pyarrow")

In [ ]:
df